# Vector Representations of Documents

http://cs.stanford.edu/~quocle/paragraph_vector.pdf

http://research.google.com/pubs/pub44894.html

http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip

Based on:

https://www.tensorflow.org/tutorials/word2vec

In [ ]:
import tensorflow as tf

In [ ]:
import os
import re
import zipfile
import shutil

HOME_DIR = 'rotten_tomatoes'
DATA_DIR = os.path.join(HOME_DIR, 'data')

print('Unpacking Stanford Sentiment Treebank dataset...')

PKG_FILE = 'stanfordSentimentTreebank.zip'
PKG_PATH = os.path.join(DATA_DIR, PKG_FILE)

from tensorflow.contrib.learn.python.learn.datasets.base import maybe_download
maybe_download(PKG_FILE, DATA_DIR, 'http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip')

DATAFILE_PATTERN = re.compile(r'^stanfordSentimentTreebank/.+\.txt$')

def extract(zip_file, filename, dst_path):
    print('Extracting', filename)
    dst_file = os.path.join(dst_path, os.path.basename(filename))
    with open(dst_file, 'wb') as fout:
        fin = zip_file.open(filename)
        shutil.copyfileobj(fin, fout)

with zipfile.ZipFile(PKG_PATH) as f:
    files = [name for name in f.namelist() if DATAFILE_PATTERN.match(name)]
    for filename in files:
        extract(f, filename, DATA_DIR)

In [ ]:
def show(filename, n=10):
    with open(filename) as f:
        for _ in range(n):
            print(next(f).strip())

In [ ]:
SENTENCES_FILE = os.path.join(DATA_DIR, 'datasetSentences.txt')
show(SENTENCES_FILE)

In [ ]:
PHRASES_FILE = os.path.join(DATA_DIR, 'dictionary.txt')
show(PHRASES_FILE)

In [ ]:
LABELS_FILE = os.path.join(DATA_DIR, 'sentiment_labels.txt')
show(LABELS_FILE)

In [ ]:
SPLIT_FILE = os.path.join(DATA_DIR, 'datasetSplit.txt')
show(SPLIT_FILE)

In [ ]:
sentences = dict()

def text_fix(txt):
    if '-LRB-' in txt:
        txt = txt.replace('-LRB-', '(')
    if '-RRB-' in txt:
        txt = txt.replace('-RRB-', ')')
    if 'Ã¡' in txt:
        txt = txt.replace('Ã¡', 'á')
    if 'Ã ' in txt:
        txt = txt.replace('Ã ', 'à')
    if 'Ã¢' in txt:
        txt = txt.replace('Ã¢', 'â')
    if 'Ã£' in txt:
        txt = txt.replace('Ã£', 'ã')
    if 'Ã©' in txt:
        txt = txt.replace('Ã©', 'é')
    if 'Ã¨' in txt:
        txt = txt.replace('Ã¨', 'è')
    if 'Ã­' in txt:
        txt = txt.replace('Ã­', 'í')
    if 'Ã¯' in txt:
        txt = txt.replace('Ã¯', 'ï')
    if 'Ã³' in txt:
        txt = txt.replace('Ã³', 'ó')
    if 'Ã´' in txt:
        txt = txt.replace('Ã´', 'ô')
    if 'Ã¶' in txt:
        txt = txt.replace('Ã¶', 'ö')
    if 'Ã»' in txt:
        txt = txt.replace('Ã»', 'û')
    if 'Ã¼' in txt:
        txt = txt.replace('Ã¼', 'ü')
    if 'Ã¦' in txt:
        txt = txt.replace('Ã¦', 'æ')
    if 'Ã§' in txt:
        txt = txt.replace('Ã§', 'ç')
    if 'Ã±' in txt:
        txt = txt.replace('Ã±', 'ñ')
    if '2Â' in txt:
        txt = txt.replace('2Â', '2')
    if '8Â' in txt:
        txt = txt.replace('8Â', '8')
    return txt

with open(SENTENCES_FILE) as f:
    f.readline() # skip header
    while True:
        line = f.readline()
        if not line:
            break
        idx, txt = line.rstrip().split('\t')
        sentences[idx] = text_fix(txt)

print("Sentences:", len(sentences))

In [ ]:
phrases = dict()

with open(PHRASES_FILE) as f:
    while True:
        line = f.readline()
        if not line:
            break
        txt, idx = line.rstrip().split('|')
        phrases[txt] = idx

print("Phrases:", len(phrases))

In [ ]:
labels = dict()

with open(LABELS_FILE) as f:
    f.readline() # skip header
    while True:
        line = f.readline()
        if not line:
            break
        idx, score = line.rstrip().split('|')
        labels[idx] = float(score)

print("Labels:", len(labels))

In [ ]:
n = 0
for txt in sentences.values():
    if txt not in phrases:
        print('Missing:', txt)
        n += 1

print('Total missing:', n)

In [ ]:
# %%time

# sentence_phrases = {s: list() for s in sentences.keys()}
# phrases_count = {phrase: 0 for phrase in phrases.keys()}

# for phrase, p in phrases.iteritems():
#     n = 0
#     for s, txt in sentences.iteritems():
#         if phrase in txt:
#             sentence_phrases[s].append(p)
#             n += 1
#     phrases_count[phrase] = n

In [ ]:
# top_phrases = sorted(phrases_count.items(), key=lambda i: i[1], reverse=True)
# top_phrases[0:30]

In [ ]:
# top_sentences = sorted(sentence_phrases.items(), key=lambda i: len(i[1]), reverse=True)
# [(s, len(p)) for s, p in top_sentences[0:30]]

In [ ]:
train = list()
valid = list()
test = list()

splits = {
    '1': train,
    '2': test,
    '3': valid,
}

with open(SPLIT_FILE) as f:
    f.readline() # skip header
    while True:
        line = f.readline()
        if not line:
            break
        idx, split = line.rstrip().split(',')
        splits[split].append(idx)

print('Train sentences:', len(train))
print('Test sentences:', len(test))
print('Validation sentences:', len(valid))

## Word2vec

In [ ]:
def clean_text(text):
    text = re.sub(r'[^A-Za-z0-9(),!?\'\`]', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip().lower()

def words(sentences_list):
    return ' '.join(clean_text(sentences[s]) for s in sentences_list).split()

words(train[0:1])

In [ ]:
import collections

VOCABULARY_SIZE = 50000

class Corpus:
    
    def __init__(self, words, vocabulary_size = VOCABULARY_SIZE):
        corpus_words = collections.Counter(words).most_common(vocabulary_size - 1)
        
        word_to_idx = dict((word, i+1) for i, (word, _) in enumerate(corpus_words))
        word_to_idx['UNK'] = 0
        word_from_idx = dict((idx, word) for word, idx in word_to_idx.items())
        
        data = list()
        unk_count = 0
        
        for word in words:
            index = word_to_idx.get(word, 0) # 0 = UNK
            if index == 0:
                unk_count += 1
            data.append(index)

        corpus_words.insert(0, ('UNK', unk_count))

        self._word_to_idx = word_to_idx
        self._word_from_idx = word_from_idx
        self.data = data
        self.words = sorted(corpus_words, key=lambda w: w[1], reverse=True)
    
    @property
    def vocabulary_size(self):
        return len(self.words)
    
    def most_common(self, n=5):
        return self.words[:n]
    
    def sample_ids(self, start=0, n=10):
        return self.data[start:(start + n)]

    def sample_words(self, start=0, n=10):
        return [self.word_from_idx(i) for i in self.sample_ids(start, n)]

    def word_to_idx(self, word):
        return self._word_to_idx.get(word, 0) # 0 = UNK
    
    def word_from_idx(self, i):
        return self._word_from_idx[i]
    
train_data = Corpus(words(train))

print('Vocabulary size:\n')
print(train_data.vocabulary_size)
print('\nMost common words (+UNK):\n')
print(train_data.most_common())
print('\nSample data:\n')
print(train_data.sample_ids(), '\n')
print(train_data.sample_words())

In [ ]:
import numpy as np

class BatchGenerator:
    
    def __init__(self, data, batch_size, num_skips, skip_window):
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window

        self.data = data
        self.data_index = 0
        
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        
        self.window_size = 2 * skip_window + 1 # [ skip_window target skip_window ]

        self.window = collections.deque(maxlen=self.window_size)
        for _ in range(self.window_size):
            self.window_forward()

    def window_forward(self):
        self.window.append(self.data[self.data_index])
        self.data_index = (self.data_index + 1) % len(self.data)

    def read(self):
        batch = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(self.batch_size, 1), dtype=np.int32)

        for i in range(self.batch_size // self.num_skips):
            center = self.skip_window
            target = center
            targets_to_avoid = [ center ]
            for j in range(self.num_skips):
                while target in targets_to_avoid:
                    target = np.random.randint(0, self.window_size)
                targets_to_avoid.append(target)
                batch[i * self.num_skips + j] = self.window[center]
                labels[i * self.num_skips + j, 0] = self.window[target]
            self.window_forward()

        return batch, labels

data_reader = BatchGenerator(train_data.data, batch_size=8, num_skips=2, skip_window=1)

print('1st Batch:\n')
batch, labels = data_reader.read()
for i in range(len(batch)):
    print(batch[i], train_data.word_from_idx(batch[i]), '->', labels[i, 0], train_data.word_from_idx(labels[i, 0]))

print('\n2nd Batch:\n')
batch, labels = data_reader.read()
for i in range(len(batch)):
    print(batch[i], train_data.word_from_idx(batch[i]), '->', labels[i, 0], train_data.word_from_idx(labels[i, 0]))

In [ ]:
class Word2vecModel:
    
    #vocabulary_size
    #embedding_size
    #num_sampled
    #examples
    
    #input_data
    #input_labels
    #loss
    #optimizer
    #embeddings
    #similarity
    
    def __init__(self,
                 vocabulary_size,
                 embedding_size,
                 num_sampled,
                 words_examples):
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.num_sampled = num_sampled
        self.words_examples = words_examples
        
        self.build_graph()
    
    def build_graph(self):
        self.input_data = tf.placeholder(tf.int32, shape=[None])
        self.input_labels = tf.placeholder(tf.int32, shape=[None, 1])
        
        embeddings = tf.Variable(
            tf.random_uniform(
                [self.vocabulary_size, self.embedding_size],
                -1.0,
                1.0))
        embed = tf.nn.embedding_lookup(embeddings, self.input_data)

        nce_weights = tf.Variable(
            tf.truncated_normal(
                [self.vocabulary_size, self.embedding_size],
                stddev=1.0 / np.sqrt(self.embedding_size)))
        nce_biases = tf.Variable(tf.zeros([self.vocabulary_size]))
    
        self.loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=self.input_labels,
                inputs=embed,
                num_sampled=self.num_sampled,
                num_classes=self.vocabulary_size))

        self.optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(self.loss)

        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        self.embeddings = embeddings / norm
        
        examples = tf.constant(self.words_examples, dtype=tf.int32)
        examples_embeddings = tf.nn.embedding_lookup(self.embeddings, examples)
        self.similarity = tf.matmul(examples_embeddings, self.embeddings, transpose_b=True)

class Word2vecTrainer:
    
    #dataset
    #batch_size
    #num_skips
    #skip_window
    #embedding_size
    #num_sampled

    def __init__(self,
                 corpus,
                 batch_size,
                 num_skips,
                 skip_window,
                 embedding_size,
                 num_sampled):
        self.corpus = corpus
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        self.embedding_size = embedding_size
        self.num_sampled = num_sampled
    
    def word_from_idx(self, i):
        return self.corpus.word_from_idx(i)

    def train(self, num_steps=100001):
        data_reader = self.data_reader()
        words_examples = self.words_examples()
        top_k = 8 # number of nearest neighbors
        
        with tf.Graph().as_default(), tf.Session() as session:
            model = self.build_model(words_examples)
            
            init = tf.global_variables_initializer()
            session.run(init)
            print('Initialized.\n')
            
            train_ops = [model.optimizer, model.loss]
            average_loss = 0
            
            for step in range(num_steps):
                batch_data, batch_labels = data_reader.read()
                feed_dict = {model.input_data: batch_data, model.input_labels: batch_labels}
                
                _, loss_val = session.run(train_ops, feed_dict=feed_dict)
                average_loss += loss_val
                
                if step % 2000 == 0:
                    if step > 0:
                        average_loss /= 2000
                    print('Average loss at step {}: {}'.format(step, average_loss))
                    average_loss = 0
                
                if step % 10000 == 0:
                    print()
                    similarity = session.run(model.similarity)
                    for i, word_idx in enumerate(words_examples):
                        word = self.word_from_idx(word_idx)
                        nearest = (-similarity[i, :]).argsort()[1:top_k+1]
                        nearest_words = (self.word_from_idx(k) for k in nearest)
                        print('Nearest to {}: {}'.format(word, ', '.join(nearest_words)))
                    print()
            
            words_embeddings = session.run(model.embeddings)
        
        return words_embeddings
    
    def data_reader(self):
        return BatchGenerator(self.corpus.data, self.batch_size, self.num_skips, self.skip_window)

    def words_examples(self, sample_size=16):
        return np.random.choice(self.corpus.vocabulary_size - 1, sample_size, replace=False)

    def build_model(self, valid_examples):
        return Word2vecModel(self.corpus.vocabulary_size, self.embedding_size, self.num_sampled, valid_examples)

In [ ]:
%%time

trainer = Word2vecTrainer(
    train_data,
    batch_size = 128,
    skip_window = 1,       # How many words to consider left and right.
    num_skips = 2,         # How many times to reuse an input to generate a label.
    embedding_size = 128,  # Dimension of the embedding vector.
    num_sampled = 64)      # Number of negative examples to sample.

words_embeddings = trainer.train()

In [ ]:
%matplotlib inline

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(words_embeddings[:plot_only,:])
labels = list(trainer.word_from_idx(i) for i in range(plot_only))

plt.figure(figsize=(18, 18))  #in inches
for i, label in enumerate(labels):
    x, y = low_dim_embs[i,:]
    plt.scatter(x, y)
    plt.annotate(
        label,
        xy=(x, y),
        xytext=(5, 2),
        textcoords='offset points',
        ha='right',
        va='bottom')

plt.show()

## Paragraph2vec

In [ ]:
for i, s in enumerate(sentences.items()):
    if i == 10:
        break
    print('{}\n\n{}\n'.format(*s))

In [ ]:
train[:10]

In [ ]:
import collections

VOCABULARY_SIZE = 50000

class DocumentCorpus:
    
    def __init__(self, documents, document_ids, vocabulary_size=VOCABULARY_SIZE):
        documents_words = dict((doc_id, clean_text(documents[doc_id]).split())
                               for doc_id in document_ids)
        
        words_flat = (word for doc_words in documents_words.values() for word in doc_words)
        words = collections.Counter(words_flat).most_common(vocabulary_size - 1)
        
        word_to_idx = dict((word, i+1) for i, (word, _) in enumerate(words))
        word_to_idx['UNK'] = 0
        word_from_idx = dict((idx, word) for word, idx in word_to_idx.items())
        
        self._word_to_idx = word_to_idx
        self._word_from_idx = word_from_idx
        
        document_vectors = list()
        document_to_idx = dict()
        document_from_idx = dict()
        
        unk_idx = word_to_idx['UNK']
        unk_count = 0
        
        for i, (doc_id, doc_words) in enumerate(documents_words.items()):
            doc_vec = list(word_to_idx.get(word, unk_idx) for word in doc_words)
            document_vectors.append(doc_vec)
            document_to_idx[doc_id] = i
            document_from_idx[i] = doc_id
            
            unk_count += doc_vec.count(unk_idx)
        
        words.insert(0, ('UNK', unk_count))
        
        self.document_vectors = document_vectors
        self.document_to_idx = document_to_idx
        self.document_from_idx = document_from_idx
        
        self.words = sorted(words, key=lambda w: w[1], reverse=True)
    
    @property
    def vocabulary_size(self):
        return len(self.words)
    
    @property
    def documents_size(self):
        return len(self.document_vectors)
    
    def most_common_words(self, n=5):
        return self.words[:n]
    
    def word_to_idx(self, word):
        return self._word_to_idx.get(word, 0) # 0 = UNK
    
    def word_from_idx(self, i):
        return self._word_from_idx[i]

    def document_form_id(self, doc_id):
        i = self.document_to_idx[doc_id]
        return self.document_vectors[i]
    
    def document_form_id_words(self, doc_id):
        doc_vec = self.document_form_id(doc_id)
        return list(self.word_from_idx(w_i) for w_i in doc_vec)

train_corpus = DocumentCorpus(sentences, train)

print('Vocabulary size:\n')
print(train_corpus.vocabulary_size)
print('\nDocuments size:\n')
print(train_corpus.documents_size)
print('\nMost common words (+UNK):\n')
print(train_corpus.most_common_words())
print('\nDocument example:\n')
print(sentences[train[0]], '\n')
print(train_corpus.document_form_id(train[0]), '\n')
print(train_corpus.document_form_id_words(train[0]))

In [ ]:
import numpy as np

class DocumentBatchGenerator:
    
    def __init__(self, document_vectors, batch_size, num_skips, skip_window):
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window

        self.document_vectors = document_vectors
        self.document_index = 0
        self.word_index = 0
        
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        
        self.window_size = 2 * skip_window + 1 # [ skip_window target skip_window ]

        self.window = collections.deque(maxlen=self.window_size)
        self.window_init()
        
    def window_init(self):
        for _ in range(self.window_size):
            self.window_next_word()
    
    def window_next_word(self):
        words = self.document_vectors[self.document_index]
        self.window.append(words[self.word_index % len(words)])
        self.word_index += 1
        
    def window_forward(self):
        words = self.document_vectors[self.document_index]
        if self.word_index >= len(words):
            self.document_index = (self.document_index + 1) % len(self.document_vectors)
            self.word_index = 0
            self.window_init()
        else:
            self.window_next_word()
    
    def read(self):
        docs = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        batch = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(self.batch_size, 1), dtype=np.int32)
        
        for i in range(self.batch_size // self.num_skips):
            center = self.skip_window
            target = center
            targets_to_avoid = [ center ]
            for j in range(self.num_skips):
                while target in targets_to_avoid:
                    target = np.random.randint(0, self.window_size)
                targets_to_avoid.append(target)
                docs[i * self.num_skips + j] = self.document_index
                batch[i * self.num_skips + j] = self.window[center]
                labels[i * self.num_skips + j, 0] = self.window[target]
            self.window_forward()

        return docs, batch, labels

data_reader = DocumentBatchGenerator(train_corpus.document_vectors, batch_size=10, num_skips=2, skip_window=1)

def print_batch():
    docs, batch, labels = data_reader.read()
    for i in range(len(batch)):
        print('{}: ({}, {}) -> ({}, {})'.format(docs[i],
                                          batch[i],
                                          train_corpus.word_from_idx(batch[i]),
                                          labels[i, 0],
                                          train_corpus.word_from_idx(labels[i, 0])))

print('1st Batch:\n')
print_batch()

print('\n2nd Batch:\n')
print_batch()

print('\n3rd Batch:\n')
print_batch()

In [ ]:
class Doc2vecModel:
    
    #documents_size
    #vocabulary_size
    #embedding_size
    #num_sampled
    
    #input_docs
    #input_data
    #input_labels
    #loss
    #optimizer
    #docs_embeddings
    #words_embeddings
    
    def __init__(self,
                 documents_size,
                 vocabulary_size,
                 embedding_size,
                 num_sampled):
        self.documents_size = documents_size
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.num_sampled = num_sampled
        
        self.build_graph()
    
    def build_graph(self):
        self.input_docs = tf.placeholder(tf.int32, shape=[None])
        self.input_data = tf.placeholder(tf.int32, shape=[None])
        self.input_labels = tf.placeholder(tf.int32, shape=[None, 1])
        
        docs_embeddings = tf.Variable(
            tf.random_uniform(
                [self.documents_size, self.embedding_size],
                -1.0,
                1.0))
        docs_lookup = tf.nn.embedding_lookup(docs_embeddings, self.input_docs)

        words_embeddings = tf.Variable(
            tf.random_uniform(
                [self.vocabulary_size, self.embedding_size],
                -1.0,
                1.0))
        words_lookup = tf.nn.embedding_lookup(words_embeddings, self.input_data)

        embed = tf.concat([docs_lookup, words_lookup], 1)
        
        nce_weights = tf.Variable(
            tf.truncated_normal(
                [self.vocabulary_size, 2 * self.embedding_size],
                stddev=1.0 / np.sqrt(2 * self.embedding_size)))
        nce_biases = tf.Variable(tf.zeros([self.vocabulary_size]))
    
        self.loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=self.input_labels,
                inputs=embed,
                num_sampled=self.num_sampled,
                num_classes=self.vocabulary_size))

        self.optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(self.loss)

        docs_norm = tf.sqrt(tf.reduce_sum(tf.square(docs_embeddings), 1, keep_dims=True))
        self.docs_embeddings = docs_embeddings / docs_norm
        
        words_norm = tf.sqrt(tf.reduce_sum(tf.square(words_embeddings), 1, keep_dims=True))
        self.words_embeddings = words_embeddings / words_norm


class NearestWordsQuery:
    
    def __init__(self, model, mapping, words):
        input_words = tf.constant(words, dtype=tf.int32)
        words_embeddings = tf.nn.embedding_lookup(model.words_embeddings, input_words)
        self.similarity = tf.matmul(words_embeddings, model.words_embeddings, transpose_b=True)
        
        self.mapping = mapping
        self.words = words

    def word_from_idx(self, i):
        return self.mapping.word_from_idx(i)
    
    def run(self, session, k=8):
        similarity = session.run(self.similarity)
        for i, word_idx in enumerate(self.words):
            word = self.word_from_idx(word_idx)
            nearest = (-similarity[i, :]).argsort()[1:k+1]
            nearest_words = ', '.join(self.word_from_idx(k) for k in nearest)
            print('Nearest to {}: {}'.format(word, nearest_words))


class Doc2vecTrainer:
    
    #dataset
    #batch_size
    #num_skips
    #skip_window
    #embedding_size
    #num_sampled

    def __init__(self,
                 corpus,
                 batch_size,
                 num_skips,
                 skip_window,
                 embedding_size,
                 num_sampled):
        self.corpus = corpus
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        self.embedding_size = embedding_size
        self.num_sampled = num_sampled
    
    def train(self, num_steps=100001):
        data_reader = self.data_reader()
        
        with tf.Graph().as_default(), tf.Session() as session:
            model = self.build_model()
            
            word_query = self.nearest_words_query(model)
            
            init = tf.global_variables_initializer()
            session.run(init)
            print('Initialized.\n')
            
            train_ops = [model.optimizer, model.loss]
            average_loss = 0
            
            for step in range(num_steps):
                batch_docs, batch_data, batch_labels = data_reader.read()
                feed_dict = {
                    model.input_docs: batch_docs,
                    model.input_data: batch_data,
                    model.input_labels: batch_labels,
                }
                
                _, loss_val = session.run(train_ops, feed_dict=feed_dict)
                average_loss += loss_val
                
                if step % 2000 == 0:
                    if step > 0:
                        average_loss /= 2000
                    print('Average loss at step {}: {}'.format(step, average_loss))
                    average_loss = 0
                
                if step % 10000 == 0:
                    print()
                    word_query.run(session)
                    print()
            
            words_embeddings = session.run(model.words_embeddings)
        
        return words_embeddings
    
    def data_reader(self):
        return DocumentBatchGenerator(self.corpus.document_vectors,
                                      self.batch_size,
                                      self.num_skips,
                                      self.skip_window)

    def build_model(self):
        return Doc2vecModel(self.corpus.documents_size,
                            self.corpus.vocabulary_size,
                            self.embedding_size,
                            self.num_sampled)
    
    def nearest_words_query(self, model):
        words = self.word_sample()
        return NearestWordsQuery(model, self.corpus, words)
        
    def word_sample(self, sample_size=5):
        return np.random.choice(self.corpus.vocabulary_size - 1,
                                sample_size,
                                replace=False)



In [ ]:
%%time

doc_trainer = Doc2vecTrainer(
    train_corpus,
    batch_size = 128,
    skip_window = 1,       # How many words to consider left and right.
    num_skips = 2,         # How many times to reuse an input to generate a label.
    embedding_size = 128,  # Dimension of the embedding vector.
    num_sampled = 64)      # Number of negative examples to sample.

words_embeddings = doc_trainer.train()

In [ ]:
%matplotlib inline

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(words_embeddings[:plot_only,:])
labels = list(doc_trainer.word_from_idx(i) for i in range(plot_only))

plt.figure(figsize=(18, 18))  #in inches
for i, label in enumerate(labels):
    x, y = low_dim_embs[i,:]
    plt.scatter(x, y)
    plt.annotate(
        label,
        xy=(x, y),
        xytext=(5, 2),
        textcoords='offset points',
        ha='right',
        va='bottom')

plt.show()